In [1]:
import pandas as pd
url = 'https://www.pro-football-reference.com/players/W/WilsRu00/fantasy/2018/'
df = pd.read_html(url)[0]
df.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 Unnamed: 2_level_0 Unnamed: 3_level_0  \
  Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
                  Rk                 G#               Date                 Tm   
0                1.0                1.0         2018-09-09                SEA   
1                2.0                2.0         2018-09-17                SEA   
2                3.0                3.0         2018-09-23                SEA   
3                4.0                4.0         2018-09-30                SEA   
4                5.0                5.0         2018-10-07                SEA   

  Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0 Unnamed: 7_level_0  \
  Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1 Unnamed: 7_level_1   
  Unnamed: 4_level_2                Opp             Result                Pos   
0                  @                DEN            L 24-27                 QB   
1                  @                CHI            L 17-24                 QB   
2                NaN                DAL            W 24-13                 QB   
3                  @                ARI            W 20-17                 QB   
4                NaN                LAR            L 31-33                 QB   

  Inside 20       ... Inside 10 Snap Counts                               \
    Passing       ...   Rushing        Off.         Def.        ST         
        Cmp  Att  ...        TD         Num     Pct  Num   Pct Num   Pct   
0       1.0  3.0  ...         0          57  100.0%    0  0.0%   0  0.0%   
1       3.0  4.0  ...         0          66  100.0%    0  0.0%   0  0.0%   
2       2.0  2.0  ...         0          69  100.0%    0  0.0%   0  0.0%   
3       0.0  1.0  ...         0          66  100.0%    0  0.0%   0  0.0%   
4       2.0  2.0  ...         0          60  100.0%    0  0.0%   0  0.0%   

  Unnamed: 28_level_0 Unnamed: 29_level_0 Unnamed: 30_level_0  
              Fantasy             Fantasy             Fantasy  
               FantPt                DKPt                FDPt  
0               20.42                22.4                22.4  
1               14.74                16.7                15.7  
2               15.58                15.6                15.6  
3                8.98                 9.0                 9.0  
4               19.92                19.9                19.9  

[5 rows x 31 columns]

In [19]:
df.columns.set_levels(['Rk', 'G#', 'Date', 'Tm', 'Away', 'Opp', 'result', 'pos', 'cmp'])

MultiIndex([( 'Unnamed: 0_level_0', 'Unnamed: 0_level_1', ...),
            ( 'Unnamed: 1_level_0', 'Unnamed: 1_level_1', ...),
            ( 'Unnamed: 2_level_0', 'Unnamed: 2_level_1', ...),
            ( 'Unnamed: 3_level_0', 'Unnamed: 3_level_1', ...),
            ( 'Unnamed: 4_level_0', 'Unnamed: 4_level_1', ...),
            ( 'Unnamed: 5_level_0', 'Unnamed: 5_level_1', ...),
            ( 'Unnamed: 6_level_0', 'Unnamed: 6_level_1', ...),
            ( 'Unnamed: 7_level_0', 'Unnamed: 7_level_1', ...),
            (          'Inside 20',            'Passing', ...),
            (          'Inside 20',            'Passing', ...),
            (          'Inside 20',            'Passing', ...),
            (          'Inside 20',            'Passing', ...),
            (          'Inside 20',            'Rushing', ...),
            (          'Inside 20',            'Rushing', ...),
            (          'Inside 20',            'Rushing', ...),
            (          'Inside 10',     

In [15]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.pro-football-reference.com'
year = 2018
maxp = 300
    
def scrape(url, year, maxp):
    # grab fantasy players
    r = requests.get(url + '/years/' + str(year) + '/fantasy.htm')
    soup = BeautifulSoup(r.content, 'html.parser')
    parsed_table = soup.find_all('table')[0]  

    df = []

    # first 2 rows are col headers
    for i,row in enumerate(parsed_table.find_all('tr')[2:]):
        if i % 10 == 0: print(i, end=' ')
        if i >= maxp: 
            print('\nComplete.')
            break

        try:
            dat = row.find('td', attrs={'data-stat': 'player'})
            name = dat.a.get_text()
            stub = dat.a.get('href')
            stub = stub[:-4] + '/fantasy/' + str(year)
            pos = row.find('td', attrs={'data-stat': 'fantasy_pos'}).get_text()

            # grab this players stats
            tdf = pd.read_html(url + stub)[0]    

            # get rid of MultiIndex, just keep last row
            tdf.columns = tdf.columns.get_level_values(-1)

            # fix the away/home column
            tdf = tdf.rename(columns={'Unnamed: 4_level_2': 'Away'})
            tdf['Away'] = [1 if r=='@' else 0 for r in tdf['Away']]

            # drop all intermediate stats
#             tdf = tdf.iloc[:,[1,2,3,4,5,-3]]

            # drop "Total" row
            tdf = tdf.query('Date != "Total"')

            # add other info
            tdf['Name'] = name
            tdf['Position'] = pos
            tdf['Season'] = year

            df.append(tdf)
        except:
            pass

    df = pd.concat(df)
    return df

In [13]:
df.to_csv('fantasy2018.csv')

In [16]:
data = scrape(url, 2017, 10)
data.head()

0 

/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ajshe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

10 
Complete.


ValueError: Plan shapes are not aligned

In [2]:
url = 'https://www.pro-football-reference.com/years/2019/receiving.htm'
df = pd.read_html(url)[0]
df.head()

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Michael Thomas*+,NOR,26,WR,16,15,185,149,80.5%,1725,11.6,9,91,49,9.3,9.3,107.8,1
1,2,Christian McCaffrey *+,CAR,23,RB,16,16,142,116,81.7%,1005,8.7,4,58,28,7.1,7.3,62.8,1
2,3,Keenan Allen*,LAC,27,WR,16,16,149,104,69.8%,1199,11.5,6,63,45,8.0,6.5,74.9,0
3,4,DeAndre Hopkins*+,HOU,27,WR,15,15,150,104,69.3%,1165,11.2,7,68,43,7.8,6.9,77.7,0
4,5,Julian Edelman,NWE,33,WR,16,13,153,100,65.4%,1117,11.2,6,54,44,7.3,6.3,69.8,3


In [22]:
def get_links(self):
    url = self.home_url + '/years/'
    self.driver.get(url)
    page = BeautifulSoup(self.driver.page_source, features="lxml")
    table = page.find(id = "div_years")
    rows = []
    table.find_all("th")
    for th in table.find_all("th"):
        for data in th.find_all("a", href = True):
            row = [data.get_text(), data['href']]
            rows.append(row)

    links = pd.DataFrame(rows, columns=['year', 'link'])
    self.links = links
    
links = get_links('https://www.pro-football-reference.com')

AttributeError: 'str' object has no attribute 'home_url'

In [21]:
from bs4 import BeautifulSoup
import requests

years = 'list of links'
url = 'https://www.pro-football-reference.com'
def get_data_tables(url, category):
     # grab fantasy players
    page = url + '/years/' + str(2019) + '/{}.htm'.format(category)
    r = requests.get(url + '/years/' + str(2019) + '/{}.htm'.format(category))
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table')[0]  
    
    tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    df.columns = df.iloc[0,:]
    df.drop(index=0,inplace=True)
    
    alt = pd.read_html(page)[0]
    return df, alt
    
#     for i,row in enumerate(parsed_table.find_all('tr')[2:]):
#         if i % 10 == 0: print(i, end=' ')
#         if i >= maxp: 
#             print('\nComplete.')
#             break
#         try:
            
    
data, alt = get_data_tables(url, 'receiving')
alt

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
0,1,Michael Thomas*+,NOR,26,WR,16,15,185,149,80.5%,1725,11.6,9,91,49,9.3,9.3,107.8,1
1,2,Christian McCaffrey *+,CAR,23,RB,16,16,142,116,81.7%,1005,8.7,4,58,28,7.1,7.3,62.8,1
2,3,Keenan Allen*,LAC,27,WR,16,16,149,104,69.8%,1199,11.5,6,63,45,8.0,6.5,74.9,0
3,4,DeAndre Hopkins*+,HOU,27,WR,15,15,150,104,69.3%,1165,11.2,7,68,43,7.8,6.9,77.7,0
4,5,Julian Edelman,NWE,33,WR,16,13,153,100,65.4%,1117,11.2,6,54,44,7.3,6.3,69.8,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,487,Vita Vea,TAM,24,NT,16,16,1,1,100.0%,1,1.0,1,1,1,1.0,0.1,0.1,0
503,488,Dwayne Washington,NOR,25,NaN,16,0,1,1,100.0%,6,6.0,0,0,6,6.0,0.1,0.4,0
504,489,Deshaun Watson *,HOU,24,QB,15,15,1,1,100.0%,6,6.0,1,1,6,6.0,0.1,0.4,10
505,490,Kerrith Whyte Jr,PIT,23,NaN,6,0,1,1,100.0%,9,9.0,0,1,9,9.0,0.2,1.5,0


In [27]:
data[data['Rk'] == "Rk"]

,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
30,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
61,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
92,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
123,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
154,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
185,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
216,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
247,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
278,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb
309,Rk,Player,Tm,Age,Pos,G,GS,Tgt,Rec,Ctch%,Yds,Y/R,TD,1D,Lng,Y/Tgt,R/G,Y/G,Fmb


In [13]:
class football_scraper:
    def __init__(self, driver, home_url):
        self.driver = driver
        self.home_url = home_url
        self.get_links()
        
    def get_links(self):
        url = self.home_url + '/years/'
        self.driver.get(url)
        page = BeautifulSoup(self.driver.page_source, features="lxml")
        table = page.find(id = "div_years")
        rows = []
        table.find_all("th")
        for th in table.find_all("th"):
            for data in th.find_all("a", href = True):
                row = [data.get_text(), data['href']]
                rows.append(row)

        links = pd.DataFrame(rows, columns=['year', 'link'])
        self.links = links
    
    def get_passing_data(self, link):
        url = self.home_url + link + 'passing.htm'
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        tables = soup.find_all("table")

        table = tables[0]
        tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                for row in table.find_all("tr")]
        df = pd.DataFrame(tab_data)
        df.columns = df.iloc[0,:]
        df.drop(index=0,inplace=True)
        df['year'] = link

        return df
    
    def giant_concat(self, collection):
        list_of_dicts = [cur_df.T.to_dict().values() for cur_df in collection]    
        giant_concat_df = pd.DataFrame(list(chain(*list_of_dicts)))
        giant_concat_df = giant_concat_df.replace(r'^\s*$', np.NaN, regex=True)
        return giant_concat_df
    
    def get_individual_passing(self, num_years, label):
        
        collection = []
        for i in range(1, num_years):
            link = self.links.loc[i]['link']
            df = self.get_passing_data(link)
            collection.append(df)
        
        combined = self.giant_concat(collection)
        combined.to_csv(label + '.csv')
    
    def get_team_stats_helper(self, link, idd):
        url = self.home_url + link
        self.driver.get(url)
        time.sleep(3)
        page = BeautifulSoup(self.driver.page_source, features="lxml")
        table = page.find(id = idd)
        tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                    for row in table.find_all("tr")]

        df = pd.DataFrame(tab_data)
        df.columns = df.iloc[0,:]
        df.drop(index=0,inplace=True)
        df['year'] = link

        return df.reset_index(drop=True)
    
    def get_team_stats(self, num_years, label, idd):
        '''
        idds:
            - "passing": passing stats 
            - "team_stats": summary of team offense (not recommended at the moment)
        '''
        collection = []
        for i in range(1, num_years):
            print("iteration: ", i)
            link = self.links.loc[i]['link']
            df = self.get_team_stats_helper(link, idd)
            collection.append(df)
            
        combined = self.giant_concat(collection)
        combined.to_csv(label + '.csv')
        
    def get_data_tables(self, category, num_years):
        
        collection = []
        for i in range(1, num_years+1):
            page = self.home_url + self.links.loc[i]['link'] + '{}.htm'.format(category)
            print(page)
            r = requests.get(page)
            soup = BeautifulSoup(r.content, 'html.parser')
            table = soup.find_all('table')[0]  

            tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                        for row in table.find_all("tr")]
            df = pd.DataFrame(tab_data)
            df.columns = df.iloc[0,:]
            df.drop(index=0,inplace=True)
            df['year'] = self.links.loc[i]['link'].split("/")[2]
            df.query("Rk != {}".format("Rk"))

            collection.append(df)


        
        tables = pd.concat(collection)
        
        return tables

    def get_advanced_data_tables_passing(self, category, num_years, tbl_id):
        
        collection = []
        for i in range(1, num_years+1):
            page = self.home_url + self.links.loc[i]['link'] + 'passing_advanced.htm'
            print(page)
            r = requests.get(page)
            soup = BeautifulSoup(r.content, 'html.parser')
            table = soup.find(id = tbl_id)
            tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                                for row in table.find_all("tr")][1:]

            df = pd.DataFrame(tab_data)
            df.columns = df.iloc[0,:]
            df.drop(index=0,inplace=True)
            df['year'] = self.links.loc[i]['link'].split("/")[2]
            df.query("Rk != {}".format("Rk"))

            collection.append(df)


        
        tables = pd.concat(collection)
        
        return tables
    


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
import requests
from itertools import chain

from bs4 import BeautifulSoup
import pandas as pd
import time

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [14]:
url = "https://www.pro-football-reference.com"
options = Options()
options.headless = True
b = webdriver.Chrome(options=options)

scraper = football_scraper(b, url)

In [15]:
data = scraper.get_data_tables("passing", 5)
data.head()

https://www.pro-football-reference.com/years/2019/passing.htm
https://www.pro-football-reference.com/years/2018/passing.htm
https://www.pro-football-reference.com/years/2017/passing.htm
https://www.pro-football-reference.com/years/2016/passing.htm
https://www.pro-football-reference.com/years/2015/passing.htm


,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD,year
1,1,Jared Goff,LAR,25,QB,16,16,9-7-0,394,626,...,86.5,50.2,22,170,6.90,6.46,3.4,1,2,2019
2,2,Jameis Winston,TAM,25,QB,16,16,7-9-0,380,626,...,84.3,59.1,47,282,7.17,6.15,7.0,2,2,2019
3,3,Matt Ryan,ATL,34,QB,15,15,7-8-0,408,616,...,92.1,60.4,48,316,6.25,6.08,7.2,3,2,2019
4,4,Tom Brady,NWE,42,QB,16,16,12-4-0,373,613,...,88.0,54.5,27,185,6.05,6.24,4.2,1,1,2019
5,5,Carson Wentz,PHI,27,QB,16,16,9-7-0,388,607,...,93.1,64.8,37,230,5.91,6.26,5.7,2,4,2019


In [16]:
data[data['year'] == '2019']

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD,year
1,1,Jared Goff,LAR,25,QB,16,16,9-7-0,394,626,...,86.5,50.2,22,170,6.90,6.46,3.4,1,2,2019
2,2,Jameis Winston,TAM,25,QB,16,16,7-9-0,380,626,...,84.3,59.1,47,282,7.17,6.15,7.0,2,2,2019
3,3,Matt Ryan,ATL,34,QB,15,15,7-8-0,408,616,...,92.1,60.4,48,316,6.25,6.08,7.2,3,2,2019
4,4,Tom Brady,NWE,42,QB,16,16,12-4-0,373,613,...,88.0,54.5,27,185,6.05,6.24,4.2,1,1,2019
5,5,Carson Wentz,PHI,27,QB,16,16,9-7-0,388,607,...,93.1,64.8,37,230,5.91,6.26,5.7,2,4,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,98,Emmanuel Sanders,2TM,32,,17,16,,1,1,...,158.3,,0,0,35.00,55.00,0.0,,,2019
102,99,Steven Sims,WAS,22,,16,2,,0,1,...,39.6,,0,0,0.00,0.00,0.0,,,2019
103,100,Courtland Sutton *,DEN,24,WR,16,14,,1,1,...,118.7,100.0,0,0,38.00,38.00,0.0,,,2019
104,101,Alex Tanney,NYG,32,,1,0,,1,1,...,79.2,1.6,0,0,1.00,1.00,0.0,,,2019


In [12]:
links = scraper.links
links.loc[0]['link'].split("/")[2]
links

,year,link
0,2020,/years/2020/
1,2019,/years/2019/
2,2018,/years/2018/
3,2017,/years/2017/
4,2016,/years/2016/
...,...,...
96,1924,/years/1924/
97,1923,/years/1923/
98,1922,/years/1922/
99,1921,/years/1921_APFA/


In [50]:
collection = []
num_years = 2
url = "https://www.pro-football-reference.com"
for i in range(1, 5):
    page = url + links.loc[i]['link'] + '{}.htm'.format("passing")
    print(page)
    r = requests.get(page)
    soup = BeautifulSoup(r.content, 'html.parser')
    table = soup.find_all('table')[0]  

    tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                for row in table.find_all("tr")]
    df = pd.DataFrame(tab_data)
    df.columns = df.iloc[0,:]
    df.drop(index=0,inplace=True)

    df.query("Rk != {}".format("Rk"))

    collection.append(df)

# data = pd.concat(collection)
# data.head()
    
    

https://www.pro-football-reference.com/years/2019//passing.htm
https://www.pro-football-reference.com/years/2018//passing.htm
https://www.pro-football-reference.com/years/2017//passing.htm
https://www.pro-football-reference.com/years/2016//passing.htm


In [52]:
df = pd.concat(collection)

In [53]:
df

,Rk,Player,Tm,Age,Pos,G,GS,QBrec,Cmp,Att,...,Y/G,Rate,QBR,Sk,Yds,NY/A,ANY/A,Sk%,4QC,GWD
1,1,Jared Goff,LAR,25,QB,16,16,9-7-0,394,626,...,289.9,86.5,50.2,22,170,6.90,6.46,3.4,1,2
2,2,Jameis Winston,TAM,25,QB,16,16,7-9-0,380,626,...,319.3,84.3,59.1,47,282,7.17,6.15,7.0,2,2
3,3,Matt Ryan,ATL,34,QB,15,15,7-8-0,408,616,...,297.7,92.1,60.4,48,316,6.25,6.08,7.2,3,2
4,4,Tom Brady,NWE,42,QB,16,16,12-4-0,373,613,...,253.6,88.0,54.5,27,185,6.05,6.24,4.2,1,1
5,5,Carson Wentz,PHI,27,QB,16,16,9-7-0,388,607,...,252.4,93.1,64.8,37,230,5.91,6.26,5.7,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,92,Jon Ryan,SEA,35,,16,0,,0,1,...,0.0,39.6,6.3,0,0,0.00,0.00,0.0,,
96,93,Willie Snead,NOR,24,WR,15,4,,1,1,...,3.3,158.3,100.0,0,0,50.00,70.00,0.0,,
97,94,Justin Tucker*+,BAL,27,,16,0,,0,1,...,0.0,39.6,,0,0,0.00,0.00,0.0,,
98,95,Tress Way,WAS,26,,16,0,,1,1,...,1.9,118.7,,0,0,31.00,31.00,0.0,,


In [4]:
url = "https://www.pro-football-reference.com/years/2019/passing_advanced.htm"
# options = Options()
# options.headless = True
# b = webdriver.Chrome(options=options)

# b.get(url)

# page = BeautifulSoup(b.page_source, features="lxml")
# table = page.find(id = "all_ks_passing_detailed_air_yards")
# tab_data = [[cell.text for cell in row.find_all(["th","td"])]
#                                     for row in table.find_all("tr")][1:]

# df = pd.DataFrame(tab_data)
# df.columns = df.iloc[0,:]
# df.drop(index=0,inplace=True)




r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
table = soup.find(id = tbl_id)
tab_data = [[cell.text for cell in row.find_all(["th","td"])]
                                    for row in table.find_all("tr")][1:]

df = pd.DataFrame(tab_data)
df.columns = df.iloc[0,:]
df.drop(index=0,inplace=True)

NameError: name 'tbl_id' is not defined

In [3]:
df

,Rk,Player,Tm,Age,Pos,G,GS,Cmp,Att,Yds,IAY,IAY/PA,CAY,CAY/Cmp,CAY/PA,YAC,YAC/Cmp
1,1,Jared Goff,LAR,25,QB,16,16,394,626,4638,4825,7.7,2388,6.1,3.8,2250,5.7
2,2,Jameis Winston,TAM,25,QB,16,16,380,626,5109,6486,10.4,3249,8.6,5.2,1860,4.9
3,3,Matt Ryan,ATL,34,QB,15,15,408,616,4466,5009,8.1,2816,6.9,4.6,1650,4.0
4,4,Tom Brady,NWE,42,QB,16,16,373,613,4057,4638,7.6,2233,6.0,3.6,1824,4.9
5,5,Carson Wentz,PHI,27,QB,16,16,388,607,4039,4878,8.0,2235,5.8,3.7,1804,4.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,98,Emmanuel Sanders,2TM,32,,17,16,1,1,35,22,22.0,22,22.0,22.0,13,13.0
102,99,Steven Sims,WAS,22,,16,2,0,1,0,5,5.0,0,,0.0,0,
103,100,Courtland Sutton *,DEN,24,WR,16,14,1,1,38,37,37.0,37,37.0,37.0,1,1.0
104,101,Alex Tanney,NYG,32,,1,0,1,1,1,1,1.0,1,1.0,1.0,0,0.0


In [ ]:
'''
all_ks_passing_detailed_air_yeards
_accuracy
_pressure
_play_type

'''